In [ ]:
# https://github.com/anjesh/Jupyter-GeoJson-Mapping
# https://www.ryanbaumann.com/blog/2016/4/3/embedding-mapbox-plots-in
#https://github.com/jwass/geojsonio.py-jupyter-notebooks
# http://jupyter-gmaps.readthedocs.io/en/latest/gmaps.html
#https://github.com/planetlabs/planet-client-python

# Setup 
Let's setup our environment. We'll pull in the the usual gis suspects and setup a leaflet map, read our API keys from a json file, and setup our Planet client

In [1]:
# See requirements.txt to set up your dev environment.
import sys
import os
import json
import scipy
import urllib
import datetime 
import urllib3
import rasterio
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
from osgeo import gdal
from planet import api
from planet.api import filters
from traitlets import link
import rasterio.tools.mask as rio_mask
from shapely.geometry import mapping, shape
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from scipy import ndimage
urllib3.disable_warnings()
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

%matplotlib inline
# will pick up api_key via environment variable PL_API_KEY
# but can be specified using `api_key` named argument
api_keys = json.load(open("apikeys.json",'r'))
client = api.ClientV1(api_key=api_keys["PLANET_API_KEY"])


# Make a slippy map to get GeoJSON

* The planet API allows you to query using a [geojson](https://en.wikipedia.org/wiki/GeoJSON) which is a special flavor of json.
* We are going to create a slippy map using leaflet and apply the Planet 2017 Q1 mosaic as the basemap. This requires our api key.
* We are going to add a special draw handler that shoves a draw region into a object so we get the geojson.
* If you don't want to do this, or need a fixed query try [geojson.io](http://geojson.io/#map=2/20.0/0.0)
* To install and run:
```
$ pip install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet
```
* [More information](https://github.com/ellisonbg/ipyleaflet)

# Querying the Planet API.
* First we'll grab our geojson area of interest (AOI) and use it to construct a query.
* We'll then build a search to search that area looking for PSScene3Band
* We have lots of products: RapidEye, PlanetScope (PS) 3 and 4 band, LandSat, and Sentinel are all possible.
* Once we have our query, we'll do the search. We will then iterate over the results, slurp up the data, and put them in a pandas data frame for easy sorting.
* We'll print the first few so we're sure it works. 

In [3]:
print AOIs[1]
myAOI = AOIs[1]["geometry"]

# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes

old = datetime.datetime(year=2013,month=1,day=1)

query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('cloud_cover', gt=0),
    filters.date_range('acquired', gt=old)
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene3Band']
)

# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
scenes = []
planet_map = {}
for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
scenes = pd.DataFrame(data=scenes)
display(scenes)
print len(scenes)

{u'geometry': {u'type': u'Polygon', u'coordinates': [[[-122.68664360046388, 45.520075090261074], [-122.68664360046388, 45.540999246978494], [-122.65625953674315, 45.540999246978494], [-122.65625953674315, 45.520075090261074], [-122.68664360046388, 45.520075090261074]]]}, u'type': u'Feature', u'properties': {}}


,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,quality_category,rows,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle
0,2017-05-03T18:15:22.572536Z,0.03,0.09,9251,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.006340,20170503_181522_0f35,PS2,...,standard,4791,0f35,489584,132.3,52.1,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-04T02:41:14Z,0,0.7
1,2017-04-27T18:15:07.721087Z,0.29,0.63,9211,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.987969,20170427_181507_102e,PS2,...,test,4767,102e,482975,133.5,50.5,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T08:42:46Z,0,0.1
2,2017-04-18T18:13:54.657796Z,0.32,0.59,9094,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942415,20170418_181354_0f25,PS2,...,test,4653,0f25,471379,135.2,47.7,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T08:15:14Z,0,0.7
3,2017-04-18T18:13:53.601975Z,0.15,0.46,9093,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942612,20170418_181353_0f25,PS2,...,test,4654,0f25,471379,135.3,47.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T05:22:23Z,0,0.7
4,2017-04-18T00:30:20.414098Z,0.00,0.32,4850,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.977989,20170418_003020_0c75,PS2,...,standard,6753,0c75,470582,259.4,25.3,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T03:52:08Z,0,1.4
5,2017-04-18T00:30:21.183098Z,0.00,0.21,4848,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.977809,20170418_003021_0c75,PS2,...,standard,6753,0c75,470582,259.5,25.2,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T07:59:05Z,0,1.4
6,2017-04-14T18:13:09.414633Z,0.48,0.69,9169,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.973864,20170414_181309_100c,PS2,...,test,4699,100c,468057,135.8,46.3,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T05:11:19Z,0,0.8
7,2017-04-15T20:18:30.991587Z,0.01,0.01,4900,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.923703,20170415_201830_1_0d06,PS2,...,test,6567,0d06,467731,183.4,54.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T07:08:13Z,0,0.8
8,2017-04-14T18:13:27.77913Z,0.49,0.68,9151,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.964380,20170414_181327_0f35,PS2,...,test,4715,0f35,466854,135.9,46.3,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05T07:11:47Z,0,0.7
9,2016-12-16T18:17:05.322596Z,0.00,0.60,9355,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.100000,20161216_181705_0e19,PS2,...,standard,4782,0e19,330529,153.9,16.9,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06T05:26:40Z,0,0.1


52


# Cleanup
* The data we got back is good, but we need some more information
* We got back big scenes, but we only care about our area of interest. The scene may not cover the whole area of interest.
* We can use the [Shapely](http://toblerity.org/shapely/manual.html) library to quickly figure out how much each scene overlaps our AOI
* We will convert our AOI and the geometry of each scene to calculate overlap using a shapely call.
* The returned acquisition, publish, and update times are strings, we'll convert them to datatime objects so we wan search.

In [4]:
# now let's clean up the datetime stuff
# make a shapely shape from our aoi
portland = shape(myAOI)
footprints = []
overlaps = []
# go through the geometry from our api call, convert to a shape and calculate overlap area.
# also save the shape for safe keeping
for footprint in scenes["geometry"].tolist():
    s = shape(footprint)
    footprints.append(s)
    overlap = 100.0*(portland.intersection(s).area / portland.area)
    overlaps.append(overlap)
# take our lists and add them back to our dataframe
scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
scenes['footprint'] = pd.Series(footprints, index=scenes.index)
# now make sure pandas knows about our date/time columns.
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])
scenes.head()

,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
0,2017-05-03 18:15:22.572536,0.03,0.09,9251,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.006340,20170503_181522_0f35,PS2,...,0f35,489584,132.3,52.1,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-04 02:41:14,0,0.7,92.309047,POLYGON ((-122.9770533769108 45.62053675570145...
1,2017-04-27 18:15:07.721087,0.29,0.63,9211,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.987969,20170427_181507_102e,PS2,...,102e,482975,133.5,50.5,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 08:42:46,0,0.1,100.000000,POLYGON ((-122.9327180670366 45.62374216604864...
2,2017-04-18 18:13:54.657796,0.32,0.59,9094,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942415,20170418_181354_0f25,PS2,...,0f25,471379,135.2,47.7,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 08:15:14,0,0.7,100.000000,"POLYGON ((-122.7430451433118 45.558984982351, ..."
3,2017-04-18 18:13:53.601975,0.15,0.46,9093,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.942612,20170418_181353_0f25,PS2,...,0f25,471379,135.3,47.6,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 05:22:23,0,0.7,6.627072,"POLYGON ((-122.721163764836 45.62499476563094,..."
4,2017-04-18 00:30:20.414098,0.00,0.32,4850,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.977989,20170418_003020_0c75,PS2,...,0c75,470582,259.4,25.3,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 03:52:08,0,1.4,13.360027,POLYGON ((-122.6648328312626 45.51991255651767...


# Filtering our search using pandas.
* Using our dataframe we will filter the scenes to just what we want.
* First we want scenes with less than 10% clouds.
* Second we want standard quality images. Test images may not be high quality.
* Third well only look for scenes since February.
* Finally we will create a new data frame with our queries and print the results. 

In [5]:
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.1
good = scenes['quality_category']=="standard"
recent = scenes["acquired"] > datetime.date(year=2017,month=1,day=1)
partial_coverage = scenes["overlap"] > 30
good_scenes = scenes[(good&clear&recent&partial_coverage)]
display(good_scenes)
print len(good_scenes)
# Now let's get it down to just good, recent, clear scenes
clear = scenes['cloud_cover']<0.5
good = scenes['quality_category']=="standard"
all_time = scenes["acquired"] > datetime.date(year=2014,month=1,day=1)
full_coverage = scenes["overlap"] >= 60
all_scenes = scenes[(good&clear&all_time&full_coverage)]
display(all_scenes)
print len(all_scenes)



,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
0,2017-05-03 18:15:22.572536,0.03,0.09,9251,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.00634,20170503_181522_0f35,PS2,...,0f35,489584,132.3,52.1,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-04 02:41:14,0,0.7,92.309047,POLYGON ((-122.9770533769108 45.62053675570145...
16,2017-03-19 18:20:53.433046,0.01,0.01,9232,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.00000,20170319_182053_0e14,PS2,...,0e14,438977,141.7,37.4,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 06:14:57,0,0.1,99.787363,POLYGON ((-122.9715821946728 45.59258510637407...
19,2017-02-13 18:19:23.607619,0.01,0.01,9093,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.90000,20170213_181923_0e14,PS2,...,0e14,409944,146.0,25.0,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 11:53:23,0,0.2,100.000000,POLYGON ((-122.8446685601367 45.58925658371648...


3


,acquired,anomalous_pixels,cloud_cover,columns,epsg_code,geometry,ground_control,gsd,id,instrument,...,satellite_id,strip_id,sun_azimuth,sun_elevation,thumbnail,updated,usable_data,view_angle,overlap,footprint
0,2017-05-03 18:15:22.572536,0.03,0.09,9251,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.006340,20170503_181522_0f35,PS2,...,0f35,489584,132.3,52.1,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-04 02:41:14,0,0.7,92.309047,POLYGON ((-122.9770533769108 45.62053675570145...
16,2017-03-19 18:20:53.433046,0.01,0.01,9232,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.000000,20170319_182053_0e14,PS2,...,0e14,438977,141.7,37.4,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 06:14:57,0,0.1,99.787363,POLYGON ((-122.9715821946728 45.59258510637407...
19,2017-02-13 18:19:23.607619,0.01,0.01,9093,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.900000,20170213_181923_0e14,PS2,...,0e14,409944,146.0,25.0,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 11:53:23,0,0.2,100.000000,POLYGON ((-122.8446685601367 45.58925658371648...
24,2016-12-06 18:16:25.531670,0.00,0.21,9277,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.000000,20161206_181625_0e19,PS2,...,0e19,318421,154.6,17.9,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 05:46:22,0,0.2,99.984638,"POLYGON ((-122.7088941214832 45.6029310678321,..."
26,2016-11-30 18:17:36.797101,0.00,0.38,9172,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.967518,20161130_181736_0e20,PS2,...,0e20,312418,154.9,18.8,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-04 00:03:04,0,0.2,100.000000,"POLYGON ((-122.95250873422 45.61470153841625, ..."
27,2016-11-06 18:16:32.028126,0.00,0.33,9069,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.933001,20161106_181632_0e30,PS2,...,0e30,286503,154.0,24.3,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 02:24:58,0,0.5,100.000000,POLYGON ((-122.9604781738547 45.61330748449009...
30,2016-10-22 18:15:49.859877,0.00,0.01,9182,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.976756,20161022_181549_0e26,PS2,...,0e26,274328,151.8,28.9,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 23:50:42,0,0.9,100.000000,POLYGON ((-122.9096717675241 45.60365519223495...
33,2016-09-30 18:14:24.794496,0.00,0.13,9351,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,4.042655,20160930_181424_0e20,PS2,...,0e20,256070,146.7,36.1,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 00:56:31,0,0.8,67.977487,POLYGON ((-122.6588675458362 45.58206841630093...
40,2016-01-10 17:40:14.626622,0.00,0.15,6880,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,3.030273,20160110_174014_0c68,PS2,...,0c68,127144,142.6,13.7,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-05 23:56:42,0,1.2,100.000000,POLYGON ((-122.5208002811397 45.54330398294279...
44,2016-03-05 18:56:43.446853,0.00,0.12,3453,32610,"{u'type': u'Polygon', u'coordinates': [[[-122....",True,2.438140,20160305_185643_0b09,PS1,...,0b09,154717,153.6,35.4,https://api.planet.com/data/v1/item-types/PSSc...,2017-05-06 02:00:26,0,0.5,97.676952,POLYGON ((-122.7529989579091 45.52399992913161...


12


# Visualizing scene foot prints overlap with our AOI
* We know these scenes intersect with our AOI, but we aren't quite sure about the geometry.
* We are going to plot our scene footprints and original AOI on our slippy map.
* To do this we create GeoJson objects with properties. 

In [6]:
# first create a list of colors
colors = ["#ff0000","#00ff00","#0000ff","#ffff00","#ff00ff","#00ffff"]
# grab our scenes from the geometry/footprint geojson
footprints = good_scenes["geometry"].tolist()
# for each footprint/color combo
for footprint,color in zip(footprints,colors):
    # create the leaflet object
    feat = {'geometry':footprint,"properties":{
            'style':{'color': color,'fillColor': color,'fillOpacity': 0.2,'weight': 1}},
            'type':u"Feature"}
    # convert to geojson
    gjson = GeoJSON(data=feat)
    # add it our map
    m.add_layer(gjson)
# now we will draw our original AOI on top 
feat = {'geometry':myAOI,"properties":{
            'style':{'color': "#FFFFFF",'fillColor': "#FFFFFF",'fillOpacity': 0.5,'weight': 1}},
            'type':u"Feature"}
gjson = GeoJSON(data=feat)
m.add_layer(gjson)   
m 


# Let's see what we got. 
* The API returns a handy thumbnail link.
* Let's tell jupyter to show it.
* You may need to login to planet explorer to have auth. 
    * If this is the case just print the urls and paste them into your browser.

In [9]:
imgs = []
# loop through our thumbnails and add display them
for img in good_scenes["thumbnail"].tolist():
    imgs.append(Image(url=img))
    print img
display(*imgs)

https://api.planet.com/data/v1/item-types/PSScene3Band/items/20170503_181522_0f35/thumb
https://api.planet.com/data/v1/item-types/PSScene3Band/items/20170319_182053_0e14/thumb
https://api.planet.com/data/v1/item-types/PSScene3Band/items/20170213_181923_0e14/thumb


# Product Activation and Downloading
* Full resolution uncompressed satellite images are *big* and there are lots of ways to view them.
* For this reason Planet generally keeps images in their native format and only processes them on customer requests. There is some caching of processed scenes, but this is the exception not the rule.
* All images must be activated prior to downloading and this can take some time based on demand.
* Additionally we need to determine what sort of product we want to download. Generally speaking there are three kinds of scenes:
    * Analytic - multi-band full resolution images that have not been processed. These are like raw files for DSLR camers.
    * Visual - these are color corrected rectified tifs. If you are just starting out this is your best call.
    * UDM - Usable data mask. This mask can be used to find bad pixels and columns and to mask out areas with clouds.
    

In [10]:
def get_products(client, scene_id, asset_type='PSScene3Band'):    
    """
    Ask the client to return the available products for a 
    given scene and asset type. Returns a list of product 
    strings
    """
    out = client.get_assets_by_id(asset_type,scene_id)
    temp = out.get()
    return temp.keys()

def activate_product(client, scene_id, asset_type="PSScene3Band",product="analytic"):
    """
    Activate a product given a scene, an asset type, and a product.
    
    On success return the return value of the API call and an activation object
    """
    temp = client.get_assets_by_id(asset_type,scene_id)  
    products = temp.get()
    if( product in products.keys() ):
        return client.activate(products[product]),products[product]
    else:
        return None 

def download_and_save(client,product):
    """
    Given a client and a product activation object download the asset. 
    This will save the tiff file in the local directory and return its 
    file name. 
    """
    out = client.download(product)
    fp = out.get_body()
    fp.write()
    return fp.name

def scenes_are_active(scene_list):
    """
    Check if all of the resources in a given list of
    scene activation objects is read for downloading.
    """
    retVal = True
    for scene in scene_list:
        if scene["status"] != "active":
            print "{} is not ready.".format(scene)
            return False
    return True

# Scenes ACTIVATE!
* Given our good scenes list we will convert the data frame "id" column into a list and activate every item in that list. 
* For this example we are going to default to using a 3Band visual product but I have included some four band methods to help you out.
* Activation usually takes about 5-15 minutes so get some coffee.

In [11]:
to_get = good_scenes["id"].tolist()
activated = []
# for each scene to get
for scene in to_get:
    # get the product 
    product_types = get_products(client,scene)
    for p in product_types:
        # if there is a visual product
        if p == "visual": # p == "basic_analytic_dn"
            print "Activating {0} for scene {1}".format(p,scene)
            # activate the product
            _,product = activate_product(client,scene,product=p)
            activated.append(product)

Activating visual for scene 20170503_181522_0f35
Activating visual for scene 20170319_182053_0e14
Activating visual for scene 20170213_181923_0e14


# Download Scenes
* In this section we will see if our scenes have been activated.
* If they are activated the client object will have its status flag set to active.
* Once that is done we will then save the scenes to the local directory.
* A smart engineer would set a path variable to store these files and check if the asset has already been downloaded prior to downloading

In [15]:
tiff_files = []
asset_type = "_3B_Visual"
# check if our scenes have been activated
if scenes_are_active(activated):
    for to_download,name in zip(activated,to_get):
        # create the product name
        name = name + asset_type + ".tif"
        # if the product exists locally
        if( os.path.isfile(name) ):
            # do nothing 
            print "We have scene {0} already, skipping...".format(name)
            tiff_files.append(name)
        elif to_download["status"] == "active":
            # otherwise download the product
            print "Downloading {0}....".format(name)
            fname = download_and_save(client,to_download)
            tiff_files.append(fname)
            print "Download done."
        else:
            print "Could not download, still activating"
else:
    print "Scenes aren't ready yet"

print tiff_files 
        

{u'status': u'inactive', u'md5_digest': None, u'_links': {u'_self': u'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTcwNTAzXzE4MTUyMl8wZjM1IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAidmlzdWFsIiwgImN0IjogIml0ZW0tdHlwZSJ9', u'activate': u'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMTcwNTAzXzE4MTUyMl8wZjM1IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAidmlzdWFsIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate', u'type': u'https://api.planet.com/data/v1/asset-types/visual'}, u'_permissions': [u'download'], u'type': u'visual'} is not ready.
Scenes aren't ready yet
[]


# Loading Images
* There are a varitety of ways to load tif data including Rasterio, GDAL, OpenCV, SKImage. 
* Today we are going to use rasterio and load each channel into a numpy array.
* Since the visual 3Band products are rotated we can also open a mask layer for processing.

In [13]:
def load_image4(filename):
    """Return a 4D (r, g, b, nir) numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b, g, r, nir = src.read()
            return np.dstack([r, g, b, nir])
        
def load_image3(filename):
    """Return a 3D (r, g, b) numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b,g,r,mask = src.read()
            return np.dstack([b, g, r])
        
def get_mask(filename):
    """Return a 1D mask numpy array with the data in the specified TIFF filename."""
    path = os.path.abspath(os.path.join('./', filename))
    if os.path.exists(path):
        with rasterio.open(path) as src:
            b,g,r,mask = src.read()
            return np.dstack([mask])

def rgbir_to_rgb(img_4band):
    """Convert an RGBIR image to RGB"""
    return img_4band[:,:,:3]

# Read Images and Use Matplotlib to show them.

In [14]:
img_files = []
masks = []
# load the images and masks
for fname in tiff_files:
    img_files.append(load_image3(fname))
    masks.append(get_mask(fname))
i = 0
# use matplotlib to display the map
for img,name in zip(img_files,tiff_files):
    plt.figure(i,figsize=(6,12))
    plt.imshow(img)
    plt.title(name)
    i+=1

# Quick Histogram
* Next up we'll plot the histogram of the image.

In [ ]:
import numpy.ma as ma
def plot_hist4(img_4band,title=""):
    # Plot a four band histogram
    r, g, b, nir = img_4band[:, :, 0], img_4band[:, :, 1], img_4band[:, :, 2], img_4band[:, :, 3]
    for slice_, name, color in ((r,'r', 'red'),(g,'g', 'green'),(b,'b', 'blue'), (nir, 'nir', 'magenta')):
        plt.hist(slice_.ravel(), bins=100, 
                 range=[0,img_4band.max()], 
                 label=name, color=color, histtype='step')
    plt.title(title)
    plt.legend()
    
def plot_hist3(img_3band,mask,title=""):
    # plot a three band histogramwaiter = []
    r, g, b = img_3band[:, :, 0], img_3band[:, :, 1], img_3band[:, :, 2]
    r = ma.masked_array(r,mask=mask)
    g = ma.masked_array(g,mask=mask)
    b = ma.masked_array(b,mask=mask)
    for slice_, name, color in ((r,'r', 'red'),(g,'g', 'green'),(b,'b', 'blue')):
        plt.hist(slice_.ravel(), bins=25, 
                 range=[0,img_3band.max()], 
                 label=name, color=color, histtype='step')
    plt.title(title)
    plt.legend()

In [ ]:
i = 0
for img,name,mask in zip(img_files,tiff_files,masks):
    plt.figure(i)
    plot_hist3(img,mask=mask,title=name)
    i+=1

# Decomposing Channels
* We can also decompose the channels of the image. 
* Sometimes it is useful to work just in a single channel.
* Other times channels can be used to do useful things, like filter out clouds.


In [ ]:
def plot_bands4(img,title="",i=0):
    fig = plt.figure(i)
    fig.set_size_inches(24, 3)
    r, g, b, nir = img[:, :, 0], img[:, :, 1], img[:, :, 2], img[:, :, 3]
    fig.suptitle(title)
    for i, (x, c) in enumerate(((r, 'r'), (g, 'g'), (b, 'b'), (nir, 'near-ir'))):
        a = fig.add_subplot(1, 4, i+1)
        a.set_title(c)
        plt.imshow(x)
        
def plot_bands3(img,title="",i=0):
    fig = plt.figure(i)
    fig.set_size_inches(24, 5)
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    fig.suptitle(title)
    for i, (x, c) in enumerate(((r, 'r'), (g, 'g'), (b, 'b'))):
        a = fig.add_subplot(1, 4, i+1)
        a.set_title(c)
        plt.imshow(x)        

In [ ]:
i = 0
for img,name in zip(img_files,tiff_files):
    plot_bands3(img,title=name,i=i)
    i+=1

# But all of these scenes are big, and we want downtown Portland 
* We can clip all of the scenes to the AOI we selected at the start of the notebook
* First we'll dump the geojson to a file.
* Since geospatial data is "big" we often work with files and get stuff out of memory ASAP.
* For each of our scenes we'll create a 'clip' file.
* We will use a tool called GDAL to clip the scene to our AOI
* GDAL stands for [Geospatial Data Abstraction Library](http://www.gdal.org/)
* GDAL is a C++ library that is often run from the command line, but it does have SWIG bindings.

In [2]:
aoi_file ="portland.geojson" 
# set our options for GDAL
options = ['-t_srs','EPSG:3857', # the projection we will use
           '-cutline', aoi_file, # the file we want to use for clipping
           '-tap',# align to our box./
           '-tr', '3', '3'] # the output resolution.
# write our input AOI to a geojson file.
with open(aoi_file,"w") as f:
    f.write(json.dumps(myAOI))

# create our full input and output names
clip_names = [os.path.abspath(tiff[:-4]+"_clip"+".tif") for tiff in tiff_files]
full_tif_files = [os.path.abspath("./"+tiff) for tiff in tiff_files]

# run GDAL warp over each of our input images. 
for in_file,out_file in zip(tiff_files,clip_names):
    derp = gdal.Warp(out_file,in_file,cutlineDSName=aoi_file,cropToCutline=True,dstSRS = 'EPSG:3857', options = options)

NameError: name 'json' is not defined

# Awesome, Let's take a look at what we got.

In [ ]:
clip_img_files = [load_image3(fname) for fname in clip_names]
    
i = 0
for img,name in zip(clip_img_files,clip_names):
    plt.figure(i,figsize=(6,12))
    plt.imshow(img)
    plt.title(name)
    i+=1

# Hrm... that's not right.
* You'll notice that a lot of these scenes don't fill our AOI.
* A lot of theses images were taken roughly at the same time.
* We should try to merge these scenes together to make one big scene.
* This process is called mosaicking, and GDAL can help. 
* We will call GDAL from the command line using subprocess to do this for us.

# WE NEED AN IMAGE HERE.


In [ ]:
commands = ["gdalwarp", # t
           "-t_srs","EPSG:3857",
           "-cutline",aoi_file,
           "-crop_to_cutline",
           "-tap",
            "-tr", "3", "3"
           "-overwrite"]
output_mosaic = "merged.tif"
for tiff in tiff_files:
    commands.append(tiff)
commands.append(output_mosaic)
print " ".join(commands)
subprocess.call(commands)

# Let's take a look.... looks much better

In [ ]:
merged = load_image3("./merged.tif")
plt.figure(i,figsize=(6,12))
plt.imshow(merged)
plt.title("merged")

In [ ]:
# Activate
to_get = all_scenes["id"].tolist()
activated = []
for scene in to_get:
    product_types = get_products(client,scene)
    for p in product_types:
        if p == "visual": # p == "basic_analytic_dn"
            print "Activating {0} for scene {1}".format(p,scene)
            _,product = activate_product(client,scene,product=p)
            activated.append(product)
# Download  
tiff_files = []
asset_type = "_3B_Visual"
if scenes_are_active(activated):
    for to_download,name in zip(activated,to_get):
        name = name + asset_type + ".tif"
        if( os.path.isfile(name) ):
            print "We have scene {0} already, skipping...".format(name)
            tiff_files.append(name)
        elif to_download["status"] == "active":
            print "Downloading {0}....".format(name)
            fname = download_and_save(client,to_download)
            tiff_files.append(fname)
            print "Download done."
        else:
            print "Could not download, still activating"
else:
    print "Scenes aren't ready yet"



In [ ]:
options = ['-t_srs','EPSG:3857', # the projection we will use
           '-cutline', aoi_file, # the file we want to use for clipping
           '-tap',# align to our box./
           '-tr', '3', '3'] # the output resolution.
clip_names = []
for tiff in tiff_files:
    clip_names.append(os.path.abspath(tiff[:-4]+"_clip"+".tif"))
full_tif_files = []

for tiff in tiff_files:
    full_tif_files.append(os.path.abspath("./"+tiff))

for in_file,out_file in zip(tiff_files,clip_names):
    gdal.Warp(out_file,in_file,cutlineDSName=aoi_file,cropToCutline=True,dstSRS = 'EPSG:3857', options = options)

temp_names = []
i = 0 
for in_file in clip_names:
    temp_name = "img{0}.gif".format(i) 
    command = ["convert", in_file, "-sample", "30x30%",temp_name]
    temp_names.append(temp_name)
    i += 1 
    subprocess.call(command)

magic = "wow.gif"
last_call = ["convert","-delay", "20","-loop","0", "img*.gif",magic]
subprocess.call(last_call)
print "done!"

<img src="./wow.gif">

In [ ]:
#https://mapbox.s3.amazonaws.com/playground/perrygeo/rasterio-docs/cookbook.html
#/usr/local/lib/python2.7/dist-packages/rasterio/mask.py:72: UserWarning: GeoJSON outside bounds of existing output raster. Are they in different coordinate reference systems?
#  "reference systems?")
# https://gist.github.com/sgillies/8655640
#https://geohackweek.github.io/vector/04-geopandas-intro/
# https://github.com/dwtkns/gdal-cheat-sheet
# http://gis-pdx.opendata.arcgis.com/datasets/neighborhood-boundaries
#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html
# htt;p://geospatialpython.com/2011/02/clip-raster-using-shapefile.html